In [213]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.signal import medfilt
import pickle
import glob
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [214]:
def read_annotation(annotation_path):
    lines = []
    with open(annotation_path) as f:
        lines.append(f.read().splitlines() )
    f.close()
    #lines = np.sort(lines)
    lines = np.hstack(lines)
    return lines

In [93]:
## 8 9 27 58 59 60

In [215]:
def get_annotation(path):
    label_skeleton = ['008','009','027','058','059','060']
    skeletons = os.listdir('./skeleton_npy')
    given_skeletons = []
    given_skeletons_label = []
    for i,skeleton in enumerate(skeletons):
        if skeleton[-16:-13] in label_skeleton:
            given_skeletons.append(skeleton[:-13])
            given_skeletons_label.append(skeleton[-16:-13])
    return given_skeletons, given_skeletons_label

In [216]:
X, y = get_annotation('skeleton_npy')

In [217]:
def data_split(examples, labels, train_frac):
    X_train, X_tmp, Y_train, Y_tmp = train_test_split( examples, labels,train_size=train_frac, random_state=42)
    X_val, X_test, Y_val, Y_test = train_test_split(X_tmp, Y_tmp, train_size=0.5, random_state=42)
    return X_train, X_val, X_test,  Y_train, Y_val, Y_test

In [218]:
# split data 80% train, 10% val, 10% test
X_train, X_val, X_test, y_train, y_val, y_test = data_split(X, y, train_frac=0.8) 

In [104]:
X_train[1], y_train[1]

('S015C001P015R002A009', '009')

In [200]:
# data = np.load('skeleton_npy/' + 'S015C001P015R002A009' + '.skeleton.npy', allow_pickle=True).item()

# items['Pose'].append(data['rgb_body0'])
# items['label'].append(y_train)

# 0 S008C002P030R001A059
# 0 S010C001P021R001A059
# 0 S007C003P025R002A060
# 0 S007C002P015R002A059
# 0 S007C001P017R001A059
# 0 S005C002P017R002A008
# 0 S014C002P037R001A059
# 0 S008C001P015R001A059

# data = np.load('skeleton_npy/' + 'S002C003P014R002A058' + '.skeleton.npy', allow_pickle=True).item()
# data['rgb_body0'][5]
# X_ = pd.DataFrame(data['rgb_body0'])

# for i,ID in enumerate(X_train):
#     data = np.load('skeleton_npy/' + ID + '.skeleton.npy', allow_pickle=True).item()
#     data['rgb_body0'][:, 0] /= 1920
#     data['rgb_body0'][:, 1] /= 1080
#     index = []
#     for i, x in enumerate(data['rgb_body0']):
#         if (x[:,0] >=1 ).sum() >0 or (x[:,1] >=1 ).sum() > 0:
#             index.append(i)
#     data['rgb_body0'] = np.delete(data['rgb_body0'], index, axis=0)


            

# a = np.array([[0.3,0.4],[1.2,0.3]])
# (a>=1).sum()

In [236]:
data = np.load('skeleton_npy/' + 'S002C003P014R002A058' + '.skeleton.npy', allow_pickle=True).item()
data['rgb_body0'][0][:,1]

array([614.9242, 500.6074, 381.8552, 325.013 , 423.1867, 523.9885,
       599.1812, 622.9269, 434.2078, 543.767 , 626.6938, 640.7875,
       610.3607, 714.574 , 817.6556, 848.7445, 619.5135, 724.6998,
       824.317 , 856.0753, 411.994 , 642.8584, 621.4941, 670.6653,
       636.6366])

In [14]:
# items = {
#     'pose': [],
#     'label': []
# }
# for i,ID in enumerate(X_train):
#     data = np.load('skeleton_npy/' + ID + '.skeleton.npy', allow_pickle=True).item()
#     items['pose'].append(data['rgb_body0'])
#     items['label'].append(y_train[i])

    

In [302]:
def get_data(data,label):
    items = {
        'pose': [],
        'label': []
    }
    for i,ID in enumerate(data):
        data = np.load('skeleton_npy/' + ID + '.skeleton.npy', allow_pickle=True).item()
        indexs = []
        for idx, x in enumerate(data['rgb_body0']):
            if ((x[:,0] >=1920 ).sum() >0) or ((x[:,1] >=1 ).sum() >= 1080) or np.isnan(x[:,0]).any() or np.isnan(x[:,1]).any():
                indexs.append(idx)
            data['rgb_body0'][idx][:,0] /=1920
            data['rgb_body0'][idx][:,1] /=1080

        data['rgb_body0'] = np.delete(data['rgb_body0'], indexs, axis=0)

        items['pose'].append(data['rgb_body0'])
        items['label'].append(label[i])
    return items
    

In [303]:
train_data = get_data(X_train, y_train)

In [278]:
test_data = get_data(X_test, y_test)

In [279]:
val_data = get_data(X_val, y_val)

In [266]:
len(val_data['pose']), len(val_data['label'])

(565, 565)

In [280]:
class Config():
    def __init__(self):        
        self.frame_l = 32 # the length of frames
        self.joint_n = 25 # the number of joints
        self.joint_d = 2 # the dimension of classes 
        self.data_dir = 'NTU/'
        self.save_dir = 'data/'
C = Config()

In [281]:
pickle.dump(train_data, open(C.save_dir+"train.pkl", "wb"))

In [282]:
pickle.dump(test_data, open(C.save_dir+"test.pkl", "wb"))

In [283]:
pickle.dump(val_data, open(C.save_dir+"valid.pkl", "wb"))

In [308]:
for i in range(len(train_data)):
    if (train_data['pose'][i] >=1).sum() > 0:
        print(i)

In [299]:
(train_data['pose'][310]).any()

False